# Social Network
Design a simplified social network that relies heavily on using different data structures. An idea adapted from [LeetCode's Design Twitter Problem](https://leetcode.com/problems/design-twitter/). The design of the system is leaved to the implementer's choice whether he adopts an OO (object oriented) style or the usual procedural one. However, following **the OO style involves bonus marks**!!


### Initialization

In [ ]:
"""
Libraries one "may" use
"""
import bisect           # https://docs.python.org/3/library/bisect.html
import heapq as hq      # https://docs.python.org/3/library/heapq.html
import networkx as nx   # https://networkx.org/documentation/stable/tutorial.html
import collections      # https://docs.python.org/3/library/collections.html
import itertools        # https://docs.python.org/3/library/itertools.html
import functools        # https://docs.python.org/3/library/functools.html
import random

In [ ]:
Time, mean = 0, 5
"""
Decorator wraps a function and 
sets atTime parameter on the fly.
"""
def setTime(mean):
    def outer(f):
        def event(*args, **kwargs):
            global Time
            Time += random.expovariate(1 / mean)
            return f(*args, **kwargs, atTime=Time)
        return event
    return outer

### Project Template
You may modify it (by applying some design pattern/s, for example) but maintain the same functionality.

In [ ]:
class User:
    users = dict()
    next_id = 0
    users_network = nx.DiGraph()

    def __init__(self, user_id, name, email):
        self.id = user_id
        self.name = name
        self.email = email
        self.reactedOnPosts = dict()
        self.posts_shared_withme = (
            list()
        )  # heap, criterion: Time of sharing the post
        User.users_network.add_node(self)
        self.posts = list()  # this will be a heap

    def __hash__(self):
        # hash using the id
        return self.id

    @staticmethod
    def create(name, email):
        cur_user = User(User.next_id, name, email)
        User.users[User.next_id] = cur_user
        User.next_id += 1
        return cur_user

    @staticmethod
    def get(userId):
        return User.users[userId]

    @staticmethod
    def search(userName):
        searchOutput = []
        for id, user in User.users.items():
            if userName.lower() in user.name.lower():
                searchOutput.append(id)
        return searchOutput

    def post(self, postBody, atTime):
        post = Post.create(postBody, atTime, self)
        hq.heappush(
            self.posts, post
        )  # minimum heap, criterion: time at which post is created
        return post

    def getNewsFeed(self):
        final_heap = hq.nlargest(5, self.posts)
        top5_shared = hq.nlargest(5, self.posts_shared_withme)
        final_heap = list(hq.merge(final_heap, top5_shared))  # iterable object
        final_heap = self.getfolloweesposts(final_heap, 3)
        newsfeed = hq.nlargest(5, final_heap)
        return list(newsfeed)

    def getfolloweesposts(self, final_heap, degree=3):
        for i in range(1, degree + 1):
            ithdegree_followees = nx.descendants_at_distance(
                User.users_network, self, i
            )
            for followee in ithdegree_followees:
                top5 = hq.nlargest(5, followee.posts)
                final_heap = list(hq.merge(top5, final_heap))
        return final_heap

    def share(self, post, users, atTime):
        for user in users:
            # returns the heap element containing the post if present else 
            # return empty list
            x = list(
                filter(lambda x: x == post, user.posts_shared_withme)
            )  
            if not x:
                hq.heappush(user.posts_shared_withme, post)
                for i in users:
                    post.sharedWith[i] = atTime

    def follow(self, followee):
        if (self, followee) not in User.users_network.edges:
            User.users_network.add_edge(self, followee)

    def unfollow(self, followee):
        if (self, followee) in User.users_network.edges:
            User.users_network.remove_node(followee)

    def getNotifications(self):
        react_list = self.reactedOnPosts.items()
        react_list = sorted(react_list, key=lambda x: x[1], reverse=True)
        ll = [(i, i.sharedWith[self]) for i in self.posts_shared_withme]
        shared_list = list(sorted(ll, key=lambda x: x[1], reverse=True))
        # formalizing the notification string that will be shown
        i = 0
        j = 0
        notifications_list = []
        for _ in range(5):
            st = ""
            c1 = i < len(react_list)
            c2 = j < len(shared_list)
            if c1 and c2:
                if react_list[i][1] < shared_list[j][1]:
                    st += (
                        f"User: {shared_list[j][0].user.name} shared a post"
                        f" with you, Post id: {shared_list[j][0].id}, Time"
                        f" done: {shared_list[j][1]}"
                    )
                    j += 1
                else:
                    st += (
                        f"A new reaction by User: {react_list[i][0][0].name},"
                        f" on Post id: {react_list[i][0][1].id}, Time done:"
                        f" {react_list[i][1]}"
                    )
                    i += 1
            elif c1:
                st += (
                    f"A new reaction by User: {react_list[i][0][0].name}, on"
                    f" Post id: {react_list[i][0][1].id}, Time done:"
                    f" {react_list[i][1]}"
                )
                i += 1
            elif c2:
                st += (
                    f"User: {shared_list[j][0].user.name} shared a post with"
                    f" you, Post id: {shared_list[j][0].id}, Time done:"
                    f" {shared_list[j][1]}"
                )
                j += 1
            else:
                break
            notifications_list.append(st)
        if len(notifications_list) == 0:
            notifications_list.append("You have no notifications")
        return notifications_list

    def react(self, post, atTime):
        if self not in post.reactions:
            post.add_reaction(self, atTime)
        else:
            post.remove_reaction(self)

    @property
    def followeelist(self):
        return list(User.users_network.neighbors(self))


class Post:
    posts = dict()
    next_id = 0

    def __init__(self, post_id, body, atTime, user):
        self.body = body
        self.atTime = atTime
        self.user = user
        self.reactions = dict()
        self.id = post_id
        self.sharedWith = dict()

    def __hash__(self):
        # hash using the id
        return self.id

    @staticmethod
    def create(body, atTime, user):
        post = Post(Post.next_id, body, atTime, user)
        Post.posts[Post.next_id] = post
        Post.next_id += 1
        return post

    @staticmethod
    def get(postId):
        return Post.posts[postId]

    def add_reaction(self, user, atTime):
        self.reactions[user] = atTime
        self.user.reactedOnPosts[(user, self)] = atTime


    def remove_reaction(self, user):
        del self.reactions[user]
        del self.user.reactedOnPosts[(user, self)]

    def __lt__(self, other):
        return self.atTime > other.atTime

    def __str__(self):
        # setting the reaction counter and how it is displayed
        n = len(self.reactions)
        react = "0 users"
        if n > 0:
            react = f"{list(self.reactions.keys())[0].name}"
        if n > 1:
            react += " and " + str(n - 1) + " other user"
            if n > 2:
                react += "s"
        # making sure the length of each line in the body is no longer that 50
        # adding new lines instead of spaces where it is necessary
        body = ""
        body_list = self.body.split()
        temp = ""
        for i in body_list:
            if len(temp) + len(i) < 50:
                temp += i + " "
            else:
                if temp != "":
                    temp += "\n"
                body += temp
                temp = i + " "
        body += temp + "\n"
        # Final post form
        po = (
            "#---------------------------------------------------#\n"
            f"Post by user : {self.user.name}, Time posted : {self.atTime}\n"
            f"\n{body}\n"
            f"Reacted on by {react}\n"
            "#---------------------------------------------------#"
        )

        return po

In [ ]:
# NOTE: postId param was removed from the method SocialNetwork.post
# We think that the Id should be generated by the system not given
# as param.
from typing import List  # NOQA


class SocialNetwork:
    # This is a singleton class ... where is my bonus?

    __instance = None
    def __new__(cls):
        if SocialNetwork.__instance is None:
            SocialNetwork.__instance = object.__new__(cls)
        return SocialNetwork.__instance

    def __init__(self):
        """
        Initialize your data structure/s here.
        """

    # ----------------------------------------------------------------- #
    def createUser(self, name, email):
        return User.create(name, email)

    # ----------------------------------------------------------------- #
    @setTime(mean)
    def post(self, userId, postBody, atTime) -> Post:
        """
        User composes a new post with string body at a certain time.
        """
        user = User.get(userId)
        post = user.post(postBody, atTime)
        return post

    # ----------------------------------------------------------------- #
    def getNewsFeed(self, userId) -> List[int]:
        """
        Retrieve the 5 most recent posts in the user's news feed.
        Each item in the news feed must be posted by
        the user himself, user's followees (1st degree relation)
        or followees of followees up to 3rd degree. Also, (if implemented)
        include shared posts with a user in his/her news feed.
        Posts must be ordered from most recent to least recent.
        """
        user = User.get(userId)
        newsfeed = user.getNewsFeed()
        for post in newsfeed:
            print(post)
        return newsfeed

    # ----------------------------------------------------------------- #
    def follow(self, followerId, followeeId) -> None:
        """
        Follower follows a followee.
        If the operation is invalid, it should be a no-op.
        """
        # the user is the followee
        if followerId == followeeId:
            return  # no operation
        # return value of follwerId(userId bardo) in the dict user this is me 
        # the user
        follower = User.get(
            followerId
        )  
        followee = User.get(followeeId)  # The person I want to follow
        follower.follow(
            followee
        )  # add the new person to my list of people I follow

    # ----------------------------------------------------------------- #
    def unfollow(self, followerId, followeeId) -> None:
        """
        Follower unfollows a followee.
        If the operation is invalid, it should be a no-op.
        """
        if followerId == followeeId:
            return  # no operation
        follower = User.get(followerId)  # me the user
        followee = User.get(followeeId)  # person I want to unfollow
        follower.unfollow(followee)  # User.unfollow(follower)

    # ----------------------------------------------------------------- #
    @setTime(mean)
    def react(self, userId, postId, atTime) -> None:
        """
        User reacts to a post at a certain time.
        If react() is called twice, the user un-reacts to the post.
        """
        user = User.get(userId)
        post = Post.get(postId)
        user.react(post, atTime)

    # --- BONUS ------------------------------------------------------- #
    def getNotifications(self, userId) -> List[str]:
        """
        Retrieve the 5 most recent user's notifications.
        When user A reacts to post of user B or shares a post
        with user B, this accounts for notification for user B.
        Notifications must be ordered from most recent to least recent.
        """
        user = User.get(userId)
        notifications = user.getNotifications()
        for notification in notifications:
            print(notification)
        return notifications

    # --- BONUS ------------------------------------------------------- #
    def search(self, userName) -> List[int]:
        """
        Provided certain name, search for the matching users' Ids.
        """
        return User.search(userName)

    # --- BONUS ------------------------------------------------------- #
    @setTime(mean)
    def share(self, userId, postId, usersIds, atTime) -> None:
        """
        User shares a post to a group of users.
        Disallow sharing a post twice with a user where this post
        has been already shared with.
        """
        user = User.get(userId)
        post = Post.get(postId)
        users = [User.get(cur_id) for cur_id in usersIds]
        user.share(post, users, atTime)

In [ ]:
import unittest
 
class TestSocialNetwork(unittest.TestCase):
    def setUp(self):
        self.sn = SocialNetwork()
        Post.posts = dict()
        User.users = dict()
        Post.next_id = 0
        User.next_id = 0
        User.users_network = nx.DiGraph()

        usernames = [
            "Abdelrahman",
            "Ahmed",
            "Fady",
            "Maria",
            "Menna",
            "enna",
            "aria",
        ]
        emails = [name + "@ds.project.org" for name in usernames]
        self.users = [
            self.sn.createUser(name, email)
            for (name, email) in zip(usernames, emails)
        ]

    def test_singleton(self):
        a = SocialNetwork()
        b = SocialNetwork()
        self.assertTrue(a is b)

    def test_post(self):
        # Good job whoever wrote User.post
        cur_user = self.users[0]
        self.assertEqual(len(Post.posts), 0)
        self.sn.post(cur_user.id, "Hi!! This is my first post.")
        self.assertEqual(len(Post.posts), 1)
        post = list(Post.posts.items())[0][1]
        self.assertIn("Hi!! This is my first post.", str(post))
        self.assertIn(post.user.name, str(post))
        self.assertIn(str(post.atTime), str(post))

    def test_follow_unfollow(self):
        # Good job whoever wrote User.follow/unfollow
        user1 = self.users[0]
        user2 = self.users[1]
        self.sn.unfollow(user1.id, user2.id)
        self.sn.follow(user1.id, user2.id)
        self.assertEqual(user1.followeelist, [user2])
        self.sn.unfollow(user1.id, user2.id)
        self.assertEqual(user1.followeelist, [])

    def test_react(self):
        # Good job whoever wrote User.react (it might be me but I'm not sure)
        # If it is, good job me <3
        user1 = self.users[0]
        user2 = self.users[1]
        post = self.sn.post(user1.id, "test post")
        self.sn.react(user2.id, post.id)
        self.assertEqual(list(post.reactions.keys()), [user2])
        self.sn.react(user2.id, post.id)
        self.assertEqual(list(post.reactions.keys()), [])

    def test_news_feed1(self):
        # I tried to crack it.
        self.sn.follow(self.users[0].id, self.users[1].id)
        self.sn.follow(self.users[1].id, self.users[2].id)
        self.sn.follow(self.users[2].id, self.users[3].id)
        self.sn.follow(self.users[3].id, self.users[4].id)
        self.sn.post(self.users[0].id, "my post")
        self.sn.post(self.users[1].id, "friend's post")
        self.sn.post(self.users[2].id, "friend's friend post")
        self.sn.post(self.users[3].id, "this should be in the newsfeed too")
        self.sn.post(self.users[4].id, "but this shouldn't")
        newsfeed = self.sn.getNewsFeed(self.users[0].id)
        posters = sorted([post.user.id for post in newsfeed])
        ids = sorted([user.id for user in self.users[:4]])
        self.assertEqual(posters, ids)

    def test_news_feed2(self):
        # But it held its own
        self.sn.follow(self.users[0].id, self.users[1].id)
        self.sn.follow(self.users[1].id, self.users[2].id)
        self.sn.follow(self.users[2].id, self.users[3].id)
        self.sn.follow(self.users[3].id, self.users[4].id)
        self.sn.post(self.users[0].id, "my post")
        self.sn.post(self.users[1].id, "friend's post")
        self.sn.post(self.users[2].id, "friend's friend post")
        self.sn.post(self.users[3].id, "this should be in the newsfeed too")
        post = self.sn.post(
            self.users[4].id, "but this shouldn't. Now it should"
        )
        self.sn.share(self.users[3].id, post.id, [self.users[0].id])
        newsfeed = self.sn.getNewsFeed(self.users[0].id)
        posters = sorted([post.user.id for post in newsfeed])
        ids = sorted([user.id for user in self.users[:5]])
        self.assertEqual(posters, ids)

    def test_news_feed3(self):
        # Really nice stuff.
        self.sn.follow(self.users[0].id, self.users[1].id)
        self.sn.follow(self.users[1].id, self.users[2].id)
        self.sn.follow(self.users[2].id, self.users[3].id)
        self.sn.follow(self.users[3].id, self.users[4].id)
        posts = list()
        posts.append(self.sn.post(self.users[0].id, "my post"))
        posts.append(self.sn.post(self.users[1].id, "friend's post"))
        posts.append(self.sn.post(self.users[2].id, "friend's friend post"))
        posts.append(
            self.sn.post(
                self.users[3].id, "this should be in the newsfeed too"
            )
        )
        posts.append(
            self.sn.post(self.users[4].id, "but this shouldn't. Now it should")
        )
        posts.append(
            self.sn.post(
                self.users[0].id, "but now, one of these six posts should die"
            )
        )
        self.sn.share(self.users[3].id, posts[4].id, [self.users[0].id])

        posts = sorted(posts)[::-1][:-1]

        newsfeed = self.sn.getNewsFeed(self.users[0].id)
        newsfeed_posts = sorted([post for post in newsfeed])[::-1]
        self.assertEqual(posts, newsfeed_posts)

    def test_search(self):
        # Good job bro
        self.assertEqual(sorted(self.sn.search("aria")), [3, 6])
        self.assertEqual(sorted(self.sn.search("enna")), [4, 5])
        self.assertEqual(sorted(self.sn.search("nobody")), [])

    # Now, the person who tested the following functions, also wrote them, so 
    # he would be pretty weird if he wrote good job for himself.
    
    def test_notifications1(self):
        post = self.sn.post(self.users[0].id, "number of notifications test")
        for i in range(1, 4):
            self.sn.react(self.users[i].id, post.id)
        notifications = self.sn.getNotifications(self.users[0].id)
        self.assertEqual(3, len(notifications))
        firstNotification = (
            f"A new reaction by User: {self.users[3].name}"
            f", on Post id: {post.id}"
            )
        self.assertEqual(
            firstNotification, notifications[0][:len(firstNotification)]
        )

    def test_notifications2(self):
        post = self.sn.post(
            self.users[0].id, "react deletion notifications test"
        )
        for i in range(1, 7):
            self.sn.react(self.users[i].id, post.id)
        self.sn.react(self.users[6].id, post.id)
        notifications = self.sn.getNotifications(self.users[0].id)
        self.assertEqual(5, len(notifications))
        firstNotification = (
            f"A new reaction by User: {self.users[5].name}"
            f", on Post id: {post.id}"
            )
        self.assertEqual(
            firstNotification, notifications[0][:len(firstNotification)]
        )

    def test_notifications3(self):
        post1 = self.sn.post(self.users[0].id, "sharing notifications test")
        post2 = self.sn.post(self.users[1].id, "sharing notifications test")
        post3 = self.sn.post(self.users[2].id, "sharing notifications test")
        for i in range(1, 6):
            self.sn.react(self.users[i].id, post1.id)
        self.sn.share(self.users[1].id, post2.id, [self.users[0].id])
        self.sn.share(self.users[2].id, post3.id, [self.users[0].id])
        notifications = self.sn.getNotifications(self.users[0].id)
        self.assertEqual(5, len(notifications))
        firstNotification = (
            f"User: {self.users[2].name} "
            f"shared a post with you, Post id: {post3.id}"
            )
        self.assertEqual(
            firstNotification, notifications[0][:len(firstNotification)]
        )


unittest.main(argv=[""], verbosity=5, exit=False, buffer=True);

test_follow_unfollow (__main__.TestSocialNetwork) ... ok
test_news_feed1 (__main__.TestSocialNetwork) ... ok
test_news_feed2 (__main__.TestSocialNetwork) ... ok
test_news_feed3 (__main__.TestSocialNetwork) ... ok
test_notifications1 (__main__.TestSocialNetwork) ... ok
test_notifications2 (__main__.TestSocialNetwork) ... ok
test_notifications3 (__main__.TestSocialNetwork) ... ok
test_post (__main__.TestSocialNetwork) ... ok
test_react (__main__.TestSocialNetwork) ... ok
test_search (__main__.TestSocialNetwork) ... ok
test_singleton (__main__.TestSocialNetwork) ... ok

----------------------------------------------------------------------
Ran 11 tests in 0.015s

OK
